In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist

In [108]:
def sd_replace(x):
    if pd.isnull(x['F3sd']):
        x['F3sd'] = x['F3'] * 0.065
    if pd.isnull(x['F4sd']):
        x['F4sd'] = x['F4'] * 0.05
    return x

data = pd.read_csv('vowel_formants_paillereau.txt', header = 0, sep=' ')
data.apply(sd_replace, axis=1)

,vowel,F1,F2,F3,F4,F1sd,F2sd,F3sd,F4sd
0,[i],273,2524,3787,4428,25,271,227.00,195.0
1,[y],273,2037,2460,3060,25,125,190.00,153.0
2,[e],400,2504,3364,3340,40,205,165.00,167.0
3,[2],400,1571,2645,3170,35,127,173.00,158.5
4,[E],624,2244,3066,3480,50,136,165.00,174.0
5,[9],585,1579,2751,3310,72,154,229.00,165.5
6,[a],805,1301,2832,3280,130,178,167.00,164.0
7,[u],288,770,2300,3080,28,70,149.50,154.0
8,[o],417,791,2530,3200,36,80,164.45,160.0
9,[O],577,1063,2510,3310,80,89,163.15,165.5


In [6]:
#We define a formant value for our input
F1=220#590
F2=2260#1780
F3=2980
input_formant=[F1,F2,F3]
input_formant

[220, 2260, 2980]

In [109]:
def vowel_prediction(input_formant, data, disp=False, formants=3):
    
    list_distances=[]
    
    for i in range(len(data)):
        data_iloc = []
        for j in range(1, formants + 1):
            data_iloc.append(data.iloc[i]['F' + str(j)])
        distance=pdist((input_formant, data_iloc), metric='euclidean')  
        #here we got the value of the array
        list_distances.append(distance[0])
   
    
    #print(list_distances)

    min_distance=min(list_distances)
    vowel_index=list_distances.index(min_distance)
    vowel=data.iloc[vowel_index]['vowel']
    
    if disp:
        print('The vowel predicted is ',vowel)
        print('Its minimun distance is ', min_distance)
    
    return vowel, min_distance


We test the function created:

In [110]:
vowel_prediction(input_formant, data)

('[E]', 413.36182697486714)

In [113]:
def vowel_prediction_sd(input_formant, data, formants=2):
    list_distances=[]
    
    fdist = abs(data['F1'] - input_formant[0])/data['F1sd']
    fdist += abs(data['F2'] - input_formant[1])/data['F2sd']
    if formants > 2:
        fdist += abs(data['F3'] - input_formant[1])/data['F2sd']
    if formants > 3:
        fdist += abs(data['F4'] - input_formant[1])/data['F2sd']
    df = pd.concat([data['vowel'], fdist], axis=1)
    min_vowel = df.loc[df[0].idxmin()].to_dict()
    return min_vowel['vowel'], min_vowel[0]

vowel_prediction_sd(input_formant, data)

('[i]', 3.094169741697417)